## $§$ 1.3 数值计算(Numerical Computation)

机器学习中数值计算主要是通过迭代来更新近似解, 而不是通过解析过程推导公式, 并且对数字计算机来说很难用有限的内存精确表示实数, 所以计算含有实数的函数是困难的, 常见的操作包括优化(找到使函数达到最大或最小值的参数值)和解线性方程组.  

### $§$ 1.3.1 上溢(Overflow)和下溢(Underflow)

处理含有实数的数学问题时最基本的困难就是需要在有限位模式(bit pattern)的数字计算机中表示无限多位的实数, 所以基本上对于所有的实数, 我们都会遇到近似误差(approximation error), 特别的, 大多数情况就是舍入误差(rounding error), 这往往会导致本来理论上可行的没有设计为最小化累积舍入误差的算法在实践上出问题.  

一种严重的舍入误差就是 **下溢(Underflow)**, 下溢发生在接近 0 的数字被舍入为 0 . 很多函数在参数为很小的数和参数为 0 时表现的很不一样, 例如我们应该避免除以 0 (有些环境会抛出异常或者使用 NaN(not-a-number)占位值(placehold value))或对 0 取对数. 

另外一种就是 **上溢(Overflow)** , 上溢发生在很大的数字被近似为 $\infty$ 或 $-\infty$ . 进一步的操作就会使它变成 NaN .

一个例子就是 `softmax` 函数, `softmax` 函数经常用于预测 Multinoulli 分布相关的概率 : $\text{softmax}(x)_i = \frac{\exp (x_i)} {\sum_{j=1}{n}\exp (x_j)}$ .  如果 $\forall x_i = c$, 则 $\text{softmax}(x) = \frac{1}{n}$, 但是如果 $c$ 是一个足够大的负数, 就会导致它的对数下溢, 这样就会使得 `softmax` 函数的分母为 0, 或者 $c$ 是足够大的正数, 就会导致上溢, 这样的行为是未定义的. 可以通过令 $\text{softmax})(z) \text{, while } z = x - \max_i x_i$ , 这样对于分子, 不可能会超过 1, 对于分母, 至少有一个 1, 并且其他都不会大于 1. 不过这仍然可能会导致分子下溢为 0, 如果对此时的 `softmax` 函数值进行取对数, 将会导致 $\log 0$ . 所以我们必须实现一个单独的函数, 并以 **数值稳定(numerically stable)** 的方式计算 $\log \text{softmax}$.

本书中的大部分算法都没有显式的考虑数值计算的细节, 低级库的作者应该在实现深度学习算法的时候把数值问题考虑在心中. **Theano** 就是这样的一个软件包, 自动检测并且稳定深度学习中许多常见的数值不稳定的表达式. 


### $§$ 1.3.2 病态条件(Poor Conditioning)

条件(Conditioning) 表征当函数的输入的细小变化会导致函数值的改变的快慢程度. 在科学计算中, 细小的输入变化可能因为舍入误差造成输出值的很大改变. 

考虑函数 $f(x) = \boldsymbol{A}^{-1} x$ , 当  $\boldsymbol{A} \in \mathbb{R}^{n \times n}$  有特征值分解, 则它的 **条件数(condition number)** 是 $\max_{i,j}\left| \frac{\lambda_i} {\lambda_j}\right|$. 当这个数很大的时候, 矩阵求逆对输入的误差特别敏感, 这种敏感性是矩阵的固有属性, 而不是在矩阵求逆过程中因为舍入误差造成的. 即使我们在乘以完全正确的矩阵, 病态条件的矩阵也会放大预先存在的误差. 在实践中, 该错误会与求逆过程本身的数值误差进一步复合(compound).

### $§$ 1.3.3 基于梯度的优化方法(Gradient-based optimization)

大部分深度学习算法都涉及某种形式的优化. 优化是指改变 $x$ 来使 $f(x)$ 的值最大或最小化. 最大化 $f(x)$ 可以由最小化 $-f(x)$来完成. 我们要最大化或最小化的函数称为 **目标函数(objective function)** 或 **准则(criterion)**, 当我们最小化它时, 我们也把它称为 **代价函数(cost function)**, **损失函数(loss function)** 或 **误差函数(error function)** (本书中将会交替使用这些术语, 不过在有些机器学习文献中会赋予这些名称特殊的意义). 我们经常用上标(superscript) $*$ 号来表示最大化或最小化函数的值. 例如 $x^* = \arg \min f(x)$ .

对于足够小的 $\epsilon \text{, } f(x - \epsilon \mathtt{sgn}(f'(x)) < f(x)$. 所以我们可以通过把 $x$ 向导数(derivative) 相反的方向移动一小步来使函数值减小. 这种技术称为 **梯度下降(gradient descent)**. 导数为 0 的点被称为 **临界点(critical point)** 或 **驻点(stationary point)** . 

![Figure 4.2][15]

> 最右边那种情况称为 **鞍点(saddle point)** .

**全局最小点(global minimum)** 可以有一个或多个. 在深度学习的背景下, 我们要优化的函数可能会含有许多不是最优的局部最小点, 或者是鞍点, 这样会使得优化变得很困难(尤其是多维情况下), 因此我们寻找使 $f$ 非常小的点, 但不一定是全局最小的点. 

![Figure 4.3][16]

在多维情况下(为了使最小最大这些术语有意义, 一般规定函数的值域必须是一维空间标量), 使用 **偏导数(partial derivative, $\frac {\partial}{\partial x_i} f(\boldsymbol{x})$)** 和 **梯度(gradient, $\nabla_{\boldsymbol{x}}f(\boldsymbol{x})$)**, **方向导数(directional derivative, 在方向 $\boldsymbol{u}$, 上的方向导数为 $\nabla_{\boldsymbol{u}}f(\boldsymbol{x}) = \nabla f(\boldsymbol{x}) \cdot \frac{\boldsymbol{u}}{|\boldsymbol{u}|}$ )** , 临界点就是梯度的所有元素都为 0 的点. 

为了使 $f$ 最小化, 我们需要找到降低最快的方向: $\min_{\boldsymbol{u}, \boldsymbol{u}^{\top}\boldsymbol{u} = 1} \boldsymbol{u}^\top \nabla _{\boldsymbol{x}} f(\boldsymbol{x}) = \min_{\boldsymbol{u}, \boldsymbol{u}^{\top}\boldsymbol{u} = 1} \lVert \boldsymbol{u} \rVert_{2} \lVert \nabla_{\boldsymbol{x}} f(\boldsymbol{x}) \rVert_2 \cos \theta = \min_{\boldsymbol{u}, \boldsymbol{u}^\top \boldsymbol{u} = 1} \cos \theta$ ($\theta$ 是 $\boldsymbol{u}$ 和梯度的夹角, 最后面是因为省略了与 $\boldsymbol{u}$ 无关的量). 也就是说 **$\boldsymbol{u}^*$为梯度相反的方向** . 这种方法称为 **最速下降法(method of steepest descent)** 或 **梯度下降(gradient descent)** .

为了最速下降, 新的点选为 $x' = x - \epsilon \nabla_x f(x)$, $\epsilon$ 就是 **学习率(learning rate)**, 一个用来表示步长的(很小的)正标量, 而且有很多种取法: 我们可以找使梯度消失的 $\epsilon$, 我们还可以用 **线搜索(linear search, 使用不同的 $\epsilon$ 来找到最小的 $f(x - \epsilon \nabla_x f(x))$ )**, 我们也可以直接求梯度为 0 的临界点而不用迭代求得 .

> 虽然梯度下降是限制于优化连续空间, 不过我们通过步长不断的移动, 可以看作是离散空间. 递增带有离散参数的目标函数被称为 **爬山(hill climbing)算法** . 

**梯度之上: Jacobian 矩阵和 Hessian 矩阵**

对于定义域和值域都是多维向量的情况, 包含所有偏导数的矩阵为 **Jacobian 矩阵**: $\boldsymbol{f}: \mathbb{R}^m \rightarrow \mathbb{R}^n, \boldsymbol{J} \in \mathbb{R}^{n \times m}, J_{i, j} = \frac{\partial}{\partial x_j} f(\boldsymbol{x})_i$ . 

我们可以认为 **二阶导(second derivative)** 作为 **曲率(curvature)** 的衡量. 

![Figure 4.4][17]

对于二阶导数, 我们可以使用 **Hessian 矩阵**: 

$\boldsymbol{H}(f)(\boldsymbol{x})_{i,j} = \frac{\partial^2}{\partial x_i \partial x_j} f(\boldsymbol{x})$

对于二阶偏导处处连续的地方, 微分算子是可交换的, 所以 $\frac{\partial^2}{\partial x_i \partial x_j} f(\boldsymbol{x}) = \frac{\partial^2}{\partial x_j \partial x_i} f(\boldsymbol{x})$
, 也就是说 Hessian 矩阵是对阵阵, $H_{i,j} = H_{j,i}$, 在深度学习的背景下, 我们遇到的大部分函数都在几乎处处有 对称 Hessian 矩阵. 因为 Hessian 矩阵是实对称阵, 所以我们可以把它分解成一组实特征值(eigenvalue)和一组特征向量(eigenvector)的正交基(orthogonal basis). 对于 $d^\top Hd$, $d$ 为 $H$ 的特征向量, 并且$H$ 对应的特征值就是特征向量 $d$ 方向上的单位向量对应的方向导数. 对于其他方向 $d$ , 二阶方向导数就是所有特征值的加权(权重都在 0 ~ 1, 并且与 $d$ 夹角越小, 权重越大)平均.  

我们在点 $\boldsymbol{x}^{(0)}$ 处作近似二阶泰勒级数(second-order Taylor series): $f(\boldsymbol{x}) \approx f(\boldsymbol{x}^{(0)}) + (\boldsymbol{x} - \boldsymbol{x}^{(0)})^\top \boldsymbol{g} + \frac{1}{2} (\boldsymbol{x} - \boldsymbol{x}^{(0)})^\top \boldsymbol{H} (\boldsymbol{x} - \boldsymbol{x}^{(0)})$, 其中 $g$ 是梯度, $H$ 是 $x^P{(0)}$ 处的 Hessian 矩阵. 如果我们使用学习率 $\epsilon$, 也就是令 $\boldsymbol{x} = \boldsymbol{x}^{(0)} - \epsilon \boldsymbol{g}$, 则  $f\left(\boldsymbol{x}^{(0)} - \epsilon \boldsymbol{g}\right) \approx f\left(\boldsymbol{x}^{(0)}\right) - \epsilon  \boldsymbol{g}^\top \boldsymbol{g} + \frac{1}{2} \epsilon^2 \boldsymbol{g}^\top \boldsymbol{H} \boldsymbol{g}$, 通过该式很容易得出, $\boldsymbol{g}^\top \boldsymbol{H} \boldsymbol{g} \le 0$ 时, $\epsilon$ 可以取任意正实数(但是实践中还是不能太大, 因为毕竟是近似泰勒级数(二阶)), 都能使函数值是下降的, 而  $\boldsymbol{g}^\top \boldsymbol{H} \boldsymbol{g} > 0$ 并且大到一定程度之后, 函数值将会上升(uphill),  并且很容易得出(二次函数) $\epsilon^* = \frac {\boldsymbol{g}^\top \boldsymbol{g}} {\boldsymbol{g}^\top \boldsymbol{H} \boldsymbol{g}}$ . 

> 最坏情况下, 也就是 $\boldsymbol{g}$ 与 $\boldsymbol{H}$ 最大特征值 $\lambda_\max$ 对应的特征向量对齐(align, 也就是共线)的时候, 最优步长 $\epsilon^* = \frac{1}{\lambda_\max}$ . 

> 对于能用二次函数(quadratic function)很好拟合的函数, Hessian 的特征值决定了学习率的量级. 

可以用二阶导判断临界点是(局部)极大值点还是极小值点, 这被称为 **二阶导测试(second derivative test)**. 一维情况是高中学过的, 对于多维情况, 也就是 $\nabla_\boldsymbol{x} f(\boldsymbol{x}) = 0$ 的时候, 如果 Hessian 矩阵在临界点是正定(positive definite), 该临界点就是极小值点(因为方向二阶导数在任意方向都是正数), 如果是负定的, 该临界点就是极大值点, 如果 Hessian 矩阵既有正特征值又有负特征值, 则这个临界点就是鞍点(saddle point, 相比于单变量情况, 多维情况给鞍点的判断带来了点积极证据, 也就是说该临界点在某个横断面(cross section)上是局部最小而在另外一个横断面上是局部最大点), 而如果是所有非零特征值是同号的并且有零特征值, 那么该临界点我们依然我无法判断是鞍点还是平坦区域(flat region)的一部分.

在多维情况下, 每一个点都有很多个(方向)二阶导数. Hessian 矩阵的条件数(condition number)很差(poor)的时候, 说明在某些方向导数变化很快而其他地方却特别慢. 梯度下降法并不知道这种改变, 所以就不知道它应该在导数长期为负的方向优先探索. 这样使得去找步长 $\epsilon$ 变得困难, 因为步长要足够小避免直接冲过(overshoot)极小值点, 但是步长很小又意味着在其他曲率较小的方向会进展得很不明显. 

我们可以使用Hessian 矩阵提供的信息来指导搜寻. 其中最简单的方法就是 **牛顿法(Newton's method)** , 牛顿法基于二阶泰勒展开 $f(\boldsymbol{x}) \approx f(\boldsymbol{x}^{(0)}) + (\boldsymbol{x} - \boldsymbol{x}^{(0)})^\top \nabla_\boldsymbol{x}f(\boldsymbol{x}^{(0)}) + \frac{1}{2} (\boldsymbol{x} - \boldsymbol{x}^{(0)})^\top \boldsymbol{H}(f) (\boldsymbol{x} - \boldsymbol{x}^{(0)})$, 求导后得到临界点 $\boldsymbol{x}^* = \boldsymbol{x}^{(0)} - \boldsymbol{H}(f)(\boldsymbol{x}^{(0)})^{-1}\nabla_{\boldsymbol{x}}f(\boldsymbol{x}^{(0)})$, 如果还函数是 **正定二次函数(positive definite quadratic function)**, 那么直接通过上次计算一次就可以得出 $x^*$, 如果函数局部可以近似为正定二次函数, 可以通过多次迭代更新计算上式(毕竟这牛顿法还得基于一个定点 $\boldsymbol{x}^{(0)}$ , 这个顶点需要迭代更新, 并且这样迭代的更新近似函数和跳到近似函数的最小点可以比梯度下降更快得到达临界点). 牛顿法在靠近极小值点(也就是盖点的 Hessian 为负定矩阵)才是有用的性质, 如果是靠近鞍点, 那就是一个危险的性质(而梯度下降不会有这样的危险, 除非梯度就是指向鞍点的). 

仅使用梯度的方法(例如梯度下降)被称为 **一阶优化算法(first-order optimazation algorithms)**, 像牛顿法这样利用 Hessian 矩阵的方法就被称为 **二阶优化算法(second-order optimazation algorithms)**. 

书上使用的这些优化方法可以应用在很大范围的函数, 但是几乎都没有保证(适用于所有), 因为书中设计到的函数族相当复杂. 在深度学习的背景下, 可以通过限制函数为 **Lipschitz 连续(continous)** 或有 其导数是Lipschitz连续的来提供一些保证. Lipschitz 连续函数指的是变化速度 **以Lipschitz 常数(constant) $\mathcal{L}$ 为界** :

$$\forall \boldsymbol{x}, \forall \boldsymbol{y}, |f(\boldsymbol{x}) - f(\boldsymbol{y})| \le \mathcal{L} \lVert \boldsymbol{x} - \boldsymbol{y} \rVert_2$$

这个性质为量化 "微小的输入改变将会带来微小的输出的改变" 提供保证(guarantee), 并且这是一个很弱(weak) 的约束(constraint), 深度学习中很多优化问题都可以通过略微的修改变得 Lipschitz 连续 . 最成功的特定优化领域或许就是 **凸优化(convex optimazation)**, 凸优化可以通过附加更强约束来提供更多的保证, 凸优化只适用于凸函数(Hessian 矩阵在处处都是半正定的), 因为这些函数没有鞍点并且所有极小值点都是全局最小值点(global minima), 但是深度学习遇到的大部分问题很难表示为凸优化的形式, 所以凸优化只是作为一些深度学习算法的子程序(subroutine), 不过凸优化中的分析思路对深度学习算法的收敛性(convergence)非常有用. 一般来说, 深度学习背景下凸优化的重要性大大降低, 更多的信息参考 *Boyd and Vandenberghe(2004)* 或 *Rockafellar(1997)* .

  [15]: ./images/1517301723997.jpg
  [16]: ./images/1517302544553.jpg
  [17]: ./images/1517399058008.jpg

### $§$ 1.3.4 约束优化(Constrained Optimization)

有时候我们只想从 $\boldsymbol{x}$ 的某些集合 $\mathbb{S}$ (也就是说定义域的子集) 中寻找最大和最小值, 这被称为 **约束优化(constrained optimization)** , 集合 $\mathbb{S}$ 中的点被称为 **可行(feasible)点** . 例如我们想找某种意义上小的点, 我们可以加以范数约束如 $\lVert x \rVert_2 \le 1$ .

一个简单的方案就是修改梯度下降使得将约束考虑在内. 如果我们使用很小的常数步长 $\epsilon$ , 我们可以先梯度下降, 然后将结果映射(project)回 $\mathbb{S}$ . 如果我们使用线搜索, 我们只搜索生成的 $\boldsymbol{x}$ 为可行点的 $\epsilon$ . 如果可以的话, 在梯度下降或线搜索之前, 把梯度映射到可行域(feasible region)会更高效.

一个更加复杂的方案就是转化成一个解和原问题一模一样的无约束的优化问题, 例如求 $\text{minize } f(\boldsymbol{x}) , \boldsymbol{x} \in \mathbb{R}^2, \lVert \boldsymbol{x} \rVert_2  \equiv 1$, 可以转化为 $g(\theta) = f([\cos \theta, \sin \theta]^\top)$, 这样对于 $\theta$ 来说没有约束. 这种方案需要对每种情况都单独设计一个转换. 

**KKT 方法(The Karush-Kuhn-Tucker approach, Lagrange 乘子法的推广形式)** 提供了约束优化的一般解决方案. 

新函数: **广义(generalized) Lagrangian** 或 **广义 Lagrange 函数** . 首先定义 $\mathbb{S}$ 为 $m$ 个函数 $g^{(i)}$  组成的等式( **等式约束(equality constraints)** )和 $n$ 个函数 $h^{(i)}$ 组成的不等式( **不等式约束(inequality constraints)** )构成的集合 $\mathbb{S} = \left\{\boldsymbol{x} | \forall i, g^{(i)}(\boldsymbol{x}) = 0, \forall j, h^{(j)} \le 0\right\} $, 并且对于每个约束, 有变量 $\lambda_i, \alpha_i$ (这些被称为 KKT 乘子(multiplier)), 所以广义 Lagrangian 定义为: $L(\boldsymbol{x}, \boldsymbol{\lambda}, \boldsymbol{\alpha}) = f(\boldsymbol{x}) + \sum_i \lambda_i g^{(i)}(\boldsymbol{x}) + \sum_j \alpha_j h^{(j)}(\boldsymbol{x})$. 我们现在就可以将约束优化转化为无约束优化: $\min_\boldsymbol{x} \max _\boldsymbol{\lambda} \max_{\boldsymbol{\alpha}, \boldsymbol{\alpha} \ge 0} L(\boldsymbol{x, \lambda, \alpha}) \Leftrightarrow \min_{\boldsymbol{x} \in \mathbb{S}} f(\boldsymbol{x})$. 这是因为满足约束条件的时候 $\max _\boldsymbol{\lambda} \max_{\boldsymbol{\alpha}, \boldsymbol{\alpha} \ge 0} L(\boldsymbol{x, \lambda, \alpha}) = f(\boldsymbol{x})$, 当不满足约束条件的时候, $\max _\boldsymbol{\lambda} \max_{\boldsymbol{\alpha}, \boldsymbol{\alpha} \ge 0} L(\boldsymbol{x, \lambda, \alpha}) = \infty$.

为了在约束条件下求最大值, 可以应用 $\min_\boldsymbol{x} \max _\boldsymbol{\lambda} \max_{\boldsymbol{\alpha}, \boldsymbol{\alpha} \ge 0} -L(\boldsymbol{x, \lambda, \alpha})$ ( Lagrangian 相反值的最小值) 或者 $\max\boldsymbol{x} \max _\boldsymbol{\lambda} \max_{\boldsymbol{\alpha}, \boldsymbol{\alpha} \ge 0} L(\boldsymbol{x, \lambda, \alpha})$.

**////////////////////////////// 这些性质都不知道怎么来的 //////////////////////////////**

等式约束对应变量 $\lambda_i$ 的符号不重要, 可以随意选择. 对于不等式约束, 如果 $h^{(i)}(\boldsymbol{x}^*) = 0$, 则称该约束是 **活跃的(active)**, 如果不等式约束不是活跃的, 则去除这一个约束的优化问题的解(solution)跟原问题的解至少有一个相同的局部解(local solution), 一个不活跃的约束可能会排除其他的一些解 . 例如, 含有一整块平坦的相等代价区域都是全局最优点(globally optimal points)的凸问题(convex problem)中这块区域可能会会因为这些约束(constraints)被消除掉, 或者在非凸问题中, 收敛时不活跃的约束可能会排除掉更好的局部驻点(local stationary points). 不过无论不活跃的约束是否被包含, 收敛时找到的点都是驻点(stationary point). 这是因为不活跃的 $h^{(i)}$有负值, 然后 $\min_{\boldsymbol{x}} \max_{\boldsymbol{\lambda}} \max_{\boldsymbol{\alpha}, \boldsymbol{\alpha} \ge 0} L(\boldsymbol{x}, \boldsymbol{\lambda}, \boldsymbol{\alpha})$ 会有$\alpha_i = 0$, 因此我们可以观察到该解中, 有 $\boldsymbol{\alpha} \bigodot \boldsymbol{h}(\boldsymbol{x}) = \boldsymbol{0}$, 换句话说, 对于所有 $i$, 我们知道在该解中至少有一个约束 $\alpha_i \ge 0$ 或 $h^{(i)}(\boldsymbol{x}) \le 0$是活跃的. 直观的说, 我们可以说这个解是由不等式强加的边界，我们必须通过对应的 KKT 乘子影响 $\boldsymbol{x}$ 的解，或者不等式对解没有影响，我们则归零 KKT 乘子。

有一些描述约束优化问题的最优点的性质, 被称为 Karush-Kuhn-Tucker (KKT) 条件(conditions) (Karush, 1939; Kuhn and Tucker, 1951). 这些是确定一个点是最优点的必要条件(necessary conditions)而不一定是充分条件(sufficient conditions): 

* 广义 Lagrangian 的梯度为 0
* 所有关于 $\boldsymbol{x}$ 和 KKT 乘子的约束都满足
* 不等式约束显示的‘‘互补松弛性’’(complementary slackness):  $\boldsymbol{\alpha} \bigodot \boldsymbol{h}(\boldsymbol{x}) = \boldsymbol{0}$


### $§$ 1.3.5 例子: 线性最小二乘(Squares)

如果我们要找到 $f(\boldsymbol{x}) = \frac{1}{2} \lVert \boldsymbol{Ax} - \boldsymbol{b} \rVert_2^2$ 的最小值. 虽然线性代数能够高效的解决该问题, 但是我们可以使用基于梯度的优化方法来解决这个问题. 

求梯度, $\nabla_\boldsymbol{x}f(\boldsymbol{x}) = \boldsymbol{A}^\top (\boldsymbol{Ax} - \boldsymbol{b}) = \boldsymbol{A}^\top \boldsymbol{Ax} - \boldsymbol{A}^\top \boldsymbol{x}$ .

![Figure 4.5][18]

如图, 可以使用梯度下降算法, 又因为这是一个二次函数, 所以我们还可以使用牛顿法来一步达到最优点.

现在我们添加约束 $\boldsymbol{x}^\top \boldsymbol{x} \le 1$, 我们定义 Lagrangian : $L(\boldsymbol{x}, \alpha) = f(\boldsymbol{x}) + \alpha(\boldsymbol{x}^\top \boldsymbol{x} - 1)$
于是我们可以通过 $\min_{\boldsymbol{x}} \max_{\alpha, \alpha \ge 0} L(\boldsymbol{x}, \alpha)$ 来找到最小值点. 

求得 Lagrangian 的导数并找到零点, $\boldsymbol{x} = (\boldsymbol{A}^\top \boldsymbol{A} + 2\alpha \boldsymbol{I})^{-1} \boldsymbol{A}^\top \boldsymbol{b}$(矩阵求逆需要用到 Moore-Penrose 伪逆), 其中 `!\alpha$` 的量纲的选取必须使其满足约束, 观察 $\frac{\partial}{\partial \alpha} L(\boldsymbol{x}, \alpha) = \boldsymbol{x}^\top \boldsymbol{x} - 1 \le 0$ 恒成立, 所以我们只需要一直增大 $\alpha$, 并且$\boldsymbol{x}$满足其约束条件就行. 